In [ ]:
import os
from google.colab import userdata
from openai import OpenAI

# 1. 從 Colab Secrets 讀取金鑰並設定為環境變數
# Colab 的 userdata.get('密鑰名稱') 會安全地讀取你的密鑰
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 2. 初始化 OpenAI Client
# 由於我們已將金鑰設定為環境變數，OpenAI() 會自動讀取
client = OpenAI()

print("OpenAI Client 初始化完成。")

OpenAI Client 初始化完成。


In [ ]:
import re

# ==================== Part B: 現代 AI 方法 ====================
# B-1: 語意相似度計算

def ai_similarity(text1, text2, api_key=None):
    """
    使用 GPT-3.5 判斷語意相似度
    (配合新的 client 初始化方式，這裡使用全域 client 物件)
    """
    # 建立 Prompt
    prompt = f"""
請評估以下兩段文字的語意相似度。
考慮因素：
1. 主題相關性
2. 語意重疊程度
3. 表達的觀點是否一致

文字1: {text1}
文字2: {text2}

請只回答一個0-100的數字，代表相似度百分比。
"""

    try:
        # 使用全域的 client 呼叫 (SDK v1.0+ 写法)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是繁體中文語意分析專家，只能回答數字。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0,  # 設定為 0 以獲得最穩定的結果
            max_tokens=10
        )

        # v1.0+ 取得回應內容 (使用物件屬性存取)
        result_text = response.choices[0].message.content.strip()

        # 使用正規表達式提取數字
        match = re.search(r'\d+', result_text)
        if match:
            score = int(match.group())
            return max(0, min(100, score))
        else:
            print(f"解析錯誤，模型回應: {result_text}")
            return 0

    except Exception as e:
        print(f"API 呼叫發生錯誤: {e}")
        return 0

# ==================== 測試資料與執行 ====================

documents = [
    "人工智慧正在改變世界，機器學習是其核心技術",        # doc[0] - AI 核心
    "深度學習推動了人工智慧的發展，特別是在圖像識別領域",    # doc[1] - AI 發展
    "今天天氣很好，適合出去運動",                # doc[2] - 生活/運動
    "機器學習和深度學習都是人工智慧的重要分支",         # doc[3] - AI 分支
    "運動有益健康，每天都應該保持運動習慣"            # doc[4] - 健康/運動
]

print("=" * 80)
print("B-1. AI 語意相似度測試")
print("=" * 80)


print(f"測試資料庫共有 {len(documents)} 筆文本。\n")

# 測試組合 1: AI 相關文章比較 (預期相似度高)
print(f"【測試 1：AI 相關性 (預期高分)】")
print(f"文本 A: {documents[0]}")
print(f"文本 B: {documents[1]}")
score1 = ai_similarity(documents[0], documents[1], api_key=None)
print(f"-> AI 判斷相似度: {score1} 分\n")

# 測試組合 2: AI vs 生活 (預期相似度低)
print(f"【測試 2：跨領域 (預期低分)】")
print(f"文本 A: {documents[0]}")
print(f"文本 B: {documents[2]}")
score2 = ai_similarity(documents[0], documents[2], api_key=None)
print(f"-> AI 判斷相似度: {score2} 分\n")

# 測試組合 3: 運動相關文章比較 (預期相似度高)
print(f"【測試 3：運動相關性 (預期高分)】")
print(f"文本 A: {documents[2]}")
print(f"文本 B: {documents[4]}")
score3 = ai_similarity(documents[2], documents[4], api_key=None)
print(f"-> AI 判斷相似度: {score3} 分")

B-1. AI 語意相似度測試
測試資料庫共有 5 筆文本。

【測試 1：AI 相關性 (預期高分)】
文本 A: 人工智慧正在改變世界，機器學習是其核心技術
文本 B: 深度學習推動了人工智慧的發展，特別是在圖像識別領域
-> AI 判斷相似度: 75 分

【測試 2：跨領域 (預期低分)】
文本 A: 人工智慧正在改變世界，機器學習是其核心技術
文本 B: 今天天氣很好，適合出去運動
-> AI 判斷相似度: 20 分

【測試 3：運動相關性 (預期高分)】
文本 A: 今天天氣很好，適合出去運動
文本 B: 運動有益健康，每天都應該保持運動習慣
-> AI 判斷相似度: 75 分


In [ ]:
import json

# ==================== B-2: AI 文本分類 ====================

def ai_classify(text, api_key=None):
    """
    使用 GPT-3.5 進行多維度分類 (情感 + 主題)
    對比 A-2 的規則式方法，AI 能更靈活地理解上下文。
    """

    # 設計 Prompt：要求模型分析情感與主題，並強制回傳 JSON 格式以便程式解析
    prompt = f"""
請分析以下文本的情感與主題。

文本內容: "{text}"

請回傳一個 JSON 物件，包含以下欄位：
1. "sentiment": 判斷情感，只能是 "正面"、"負面" 或 "中性"。
2. "topic": 判斷主題類別 (例如：美食、科技、運動、娛樂、旅遊、生活...等，請依據內容自動判斷最適合的詞)。
3. "confidence": 信心分數 (0.0 到 1.0 之間)。

只回傳 JSON 字串，不要有其他說明。
"""

    try:
        # 使用全域 client 呼叫
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是一個精準的文本分析 API，只回傳 JSON 格式。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=100
        )

        # 解析回傳的內容
        result_text = response.choices[0].message.content.strip()

        # 嘗試將字串轉為 JSON 物件
        # 有時候模型會包含 markdown code block 標記，需簡單清理
        if result_text.startswith("```json"):
            result_text = result_text.replace("```json", "").replace("```", "")

        data = json.loads(result_text)
        return data

    except Exception as e:
        print(f"AI 分類發生錯誤: {e}")
        return {"sentiment": "未知", "topic": "未知", "confidence": 0.0}

# ==================== 測試資料與執行 ====================

test_texts = [
    "這家餐廳的牛肉麵真的太好吃了，湯頭濃郁，麵條Q彈，下次一定再來！",  # 預期：美食 / 正面
    "最新的AI技術突破讓人驚艷，深度學習模型的表現越來越好",          # 預期：科技 / 正面
    "這部電影劇情空洞，演技糟糕，完全是浪費時間",                  # 預期：娛樂 / 負面
    "每天慢跑5公里，配合適當的重訓，體能進步很多"                  # 預期：運動 / 正面
]

print("=" * 80)
print("B-2. AI 文本分類測試")
print("=" * 80)

for i, text in enumerate(test_texts, 1):
    print(f"\n測試文本 {i}: {text}")

    # 呼叫 AI 分類
    result = ai_classify(text)

    print(f"  [AI 判斷結果]")
    print(f"  情感: {result.get('sentiment')}  (信心度: {result.get('confidence')})")
    print(f"  主題: {result.get('topic')}")


B-2. AI 文本分類測試

測試文本 1: 這家餐廳的牛肉麵真的太好吃了，湯頭濃郁，麵條Q彈，下次一定再來！
  [AI 判斷結果]
  情感: 正面  (信心度: 0.9)
  主題: 美食

測試文本 2: 最新的AI技術突破讓人驚艷，深度學習模型的表現越來越好
  [AI 判斷結果]
  情感: 正面  (信心度: 0.9)
  主題: 科技

測試文本 3: 這部電影劇情空洞，演技糟糕，完全是浪費時間
  [AI 判斷結果]
  情感: 負面  (信心度: 0.95)
  主題: 電影

測試文本 4: 每天慢跑5公里，配合適當的重訓，體能進步很多
  [AI 判斷結果]
  情感: 正面  (信心度: 0.9)
  主題: 運動


In [ ]:
# ==================== B-3: AI 自動摘要 ====================

def ai_summarize(text, ratio=0.3, api_key=None):
    """
    使用 GPT-3.5 進行生成式摘要
    對比 A-3 的抽取式方法，AI 能重組句子並歸納重點。
    """

    # 將比例轉換為百分比描述，放入 Prompt
    target_percent = int(ratio * 100)

    # 設計 Prompt
    prompt = f"""
請扮演一位專業編輯，將以下文章進行摘要。

要求：
1. 保留文章的核心資訊與關鍵論點。
2. 語句必須通順流暢 (不要只是碎片化的句子)。
3. 摘要長度大約控制在原文的 {target_percent}% 左右。

文章內容:
{text}
"""

    try:
        # 使用全域 client 呼叫
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是一位精通繁體中文的專業編輯。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3, # 摘要需要一定的穩定性，但也需要一點彈性來重組句子
            max_tokens=300
        )

        # 取得摘要內容
        summary = response.choices[0].message.content.strip()
        return summary

    except Exception as e:
        print(f"AI 摘要發生錯誤: {e}")
        return "摘要生成失敗"

# ==================== 測試資料與執行 ====================

article = """人工智慧 (AI) 的發展正在深刻改變我們的生活方式。從早上起床時的智慧鬧鐘，到通勤時的路線規劃，再到工作中的各種輔助工具，AI無處不在。

在醫療領域，AI協助醫生進行疾病診斷，提高了診斷的準確率和效率。透過分析大量的醫療影像和病歷資料，AI能夠發現人眼容易忽略的細節，為患者提供更好的治療方案。

教育方面，AI個人化學習系統能夠根據每個學生的學習進度和特點，提供客製化的教學內容。這種因材施教的方式，讓學習變得更加高效和有趣。

然而，AI的快速發展也帶來了一些挑戰。首先是就業問題，許多傳統工作可能會被AI取代。其次是隱私和安全問題，AI系統需要大量數據來訓練，如何保護個人隱私成為重要議題。最後是倫理問題，AI的決策過程往往缺乏透明度，可能會產生偏見或歧視。

面對這些挑戰，我們需要在推動AI發展的同時，建立相應的法律法規和倫理準則。只有這樣，才能確保AI技術真正為人類福祉服務，創造一個更美好的未來。"""

print("=" * 80)
print("B-3. AI 自動摘要測試")
print("=" * 80)

print(f"原文長度: {len(article)} 字")
print(f"目標壓縮比: 30%\n")

# 呼叫 AI 摘要
ai_summary_result = ai_summarize(article, ratio=0.3)

print("【AI 生成的摘要結果】:")
print(ai_summary_result)
print(f"\n摘要長度: {len(ai_summary_result)} 字")


B-3. AI 自動摘要測試
原文長度: 407 字
目標壓縮比: 30%

【AI 生成的摘要結果】:
人工智慧 (AI) 的應用已深刻改變生活，涵蓋鬧鐘、路線規劃、醫療診斷和教育個人化學習等領域。AI在醫療中提高準確率和效率，教育中提供客製化教學內容。然而，AI發展也帶來就業、隱私安全和倫理挑戰，需要相應法律法規和倫理準則來確保AI技術造福人類，創造更美好未來。

摘要長度: 131 字
